In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.utils import to_categorical

# Load the ASL Alphabet dataset
data_dir = 'archive/asl_alphabet_train/asl_alphabet_train'  # Replace with the path to your dataset
classes = os.listdir(data_dir)
num_classes = len(classes)

# Initialize lists to store images and their labels
images = []
labels = []

# Load images and labels from the dataset
for class_idx, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Load images in color
        image = cv2.resize(image, (50, 50))  # Resize the image to a consistent size
        images.append(image)
        labels.append(class_idx)

# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Load pre-trained DenseNet model (excluding top layers)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(50, 50, 3))

# Build your model on top of the DenseNet base
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())  # Global Average Pooling Layer
model.add(Dense(50, activation='relu'))  # Add your custom Dense layers here
model.add(Dense(num_classes, activation='softmax'))  # Output layer

# Freeze the layers of the pre-trained DenseNet
base_model.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Saving h5 file:
model.save('FinalModel_with_DenseNet.h5')

29084464/29084464 [==============================] - 8s 0us/step
Epoch 1/10
1958/1958 [==============================] - 220s 109ms/step - loss: 0.9807 - accuracy: 0.7392 - val_loss: 0.4270 - val_accuracy: 0.8809
Epoch 2/10
1958/1958 [==============================] - 200s 102ms/step - loss: 0.3175 - accuracy: 0.9137 - val_loss: 0.2364 - val_accuracy: 0.9346
Epoch 3/10
1958/1958 [==============================] - 197s 100ms/step - loss: 0.1987 - accuracy: 0.9439 - val_loss: 0.1732 - val_accuracy: 0.9507
Epoch 4/10
1958/1958 [==============================] - 196s 100ms/step - loss: 0.1452 - accuracy: 0.9597 - val_loss: 0.1342 - val_accuracy: 0.9582
Epoch 5/10
1958/1958 [==============================] - 202s 103ms/step - loss: 0.1127 - accuracy: 0.9677 - val_loss: 0.1153 - val_accuracy: 0.9632
Epoch 6/10
1958/1958 [==============================] - 209s 107ms/step - loss: 0.0936 - accuracy: 0.9726 - val_loss: 0.1130 - val_accuracy: 0.9642
Epoch 7/10
1958/1958 [=========================